In [ ]:
# NOTE: The `ndv` package is not yet supported in Colab. Most of the `ndv` lines are commented out.
# The `matplotlib` package can be used instead (already included in the pip list).

# Pearson's correlation coefficient

In [ ]:
%pip install matplotlib
# %pip install "ndv[jupyter,vispy]"
%pip install numpy
%pip install scikit-image
%pip install scipy
%pip install tifffile
%pip install imagecodecs

## Description

In this section, we will explore how to implement in Python the **Pearson's Correlation Coefficient**, which is a common method for quantifying colocalization based on pixel intensities.

The images we will use for this section can be downloaded from the <a href="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/08_pixel_intensity_based_coloc.zip" download> <i class="fas fa-download"></i> Mander's & Pearson's Colocalization Dataset</a>.

<p class="alert alert-warning">
    <strong>Note:</strong> This notebook aims to show how to practically implement these methods but does not aim to describe when to use which method. For this exercise, we will use a single 2-channel image and without any preprocessing steps.
</p>

<p class="alert alert-info">
    <strong>Note:</strong> In this example, we will not perform any image processing steps before computing the Pearson's Correlation Coefficient. However, when conducting a real colocalization analysis you should consider applying some image processing steps to clean the images before computing the Pearson's Correlation Coefficient, such as background subtraction, flat-field correction, etc.
</p>

<p class="alert alert-info">
    <strong>Note:</strong> In this notebook we will only use a single image pair for demonstration purposes. Often, Pearson's coefficients should not be interpreted as absolute values in isolation. Instead, it's always recommended to consider them in the context of comparisons between different conditions, controls, treatments, or experimental groups. The relative changes and ratios between conditions are often more meaningful than the absolute coefficient values themselves.
</p>

## Pearson's Correlation Coefficients

The Pearson's correlation coefficient measures the **linear relationship** between pixel intensities in two fluorescence channels. 

<div align="center"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/coloc/pearsons_slide.png" alt="pearsons" width="800"></div>

<br>

It quantifies how well the intensity variations in one channel predict the intensity variations in another channel across all pixels in the image. 

The coefficient ranges from **-1 to +1**, where:
- **+1** indicates perfect positive correlation (when one channel's intensity increases, the other increases proportionally)
- **0** indicates no linear correlation
- **-1** indicates perfect negative correlation (when one channel's intensity increases, the other decreases proportionally)

<div align="center"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/coloc/pearsons_graphs.png" alt="pearsons_graphs" width="770"></div>

<br>

Pearson's correlation considers the **entire intensity range** and evaluates how intensities **co-vary** across the image. This makes it particularly useful for detecting cases where two proteins show coordinated expression levels, even if they don't necessarily occupy the exact same pixels.

### Load and Visualize the Image

Open and visualize (with ndv) the image named `cells_pearsons.tif` from the <a href="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/08_pixel_intensity_based_coloc.zip" download><i class="fas fa-download"></i> Mander's & Pearson's Colocalization Dataset</a>. This is a two-channel image of HEK293 cells where two distinct fluorescent proteins have been labeled with different fluorescent markers.

To compute Pearson's Correlation Coefficients, we need **two separate images** (channels).

What is the image shape? How do we split the channels?

### Scatter Plot

It is often useful to visualize the relationship between the two channels using a scatter plot. This can help us understand the distribution of pixel intensities.

### Calculate Pearson's Correlation Coefficients

There are several libraries in Python that alreqady implement the Pearson's Correlation Coefficient. Two examples are `scipy.stats.pearsonr` and `numpy.corrcoef`.

In [ ]:
# Calculate Pearson's correlation coefficient using scipy
pearson, p_value = pearsonr(ch1.flatten(), ch2.flatten())
print(f"Pearson's (scipy): {pearson:.4f}, p-value: {p_value:.4f}")
# Calculate Pearson's correlation coefficient using numpy
pearson_numpy = np.corrcoef(ch1.flatten(), ch2.flatten())[0, 1]
print(f"Pearson's (numpy): {pearson_numpy:.4f}")

Pearson's (scipy): 0.8771, p-value: 0.0000
Pearson's (numpy): 0.8771


### Costes Pixel Randomization Test

The **Costes pixel randomization test** is a statistical method used to validate the significance of colocalization results, particularly for Pearson's correlation coefficients. This method involves **randomly shuffling the pixel intensities of one channel and recalculating the Pearson's correlation coefficient** to create a distribution of values under the null hypothesis of no colocalization.

Below you can find an implementation of this method in Python. This function returns the observed Pearson's correlation coefficient, a list of randomized correlation coefficients, and the p-value indicating the significance of the observed correlation.

A low `p-value` (e.g. 0.0001) means that none of the `n` random translations (by default 500) produced a correlation coefficient as high as the observed one, indicating that the observed colocalization is statistically significant: the probability of getting the observed colocalization by random chance is < 0.0001 (less than 0.01%).

Let's run it on the two channels we have been working with.

In [120]:
def costes_pixel_randomization(
    channel_1: np.ndarray,
    channel_2: np.ndarray,
    n_iterations: int = 500,
    seed: int = 3,
) -> tuple[float, list[float], float]:
    """
    Perform Costes pixel randomization test for colocalization significance.

    Parameters:
    -----------
    channel_1 : np.ndarray
        Reference channel (kept unchanged)
    channel_2 : np.ndarray
        Channel to be randomized
    n_iterations : int
        Number of randomization iterations (default: 1000)
    seed : int
        Random numpy seed for reproducibility (default: 3)

    Returns:
    --------
    Tuple containing:
        - float: Observed correlation coefficient
        - List[float]: Correlation coefficients from randomized iterations
        - float: P-value (fraction of random correlations >= observed)
    """
    # Set the numpy random seed for reproducibility
    np.random.seed(seed)

    # Calculate observed correlation
    observed_correlation, _ = pearsonr(channel_1.flatten(), channel_2.flatten())

    # Initialize list to store randomized correlations
    random_correlations = []

    # Store original shape for reshaping
    shape = channel_2.shape

    # for _ in tqdm(range(n_iterations), desc="Costes pixel randomization"):
    for _ in range(n_iterations):
        # Flatten, shuffle, and reshape
        randomized_channel_2 = np.random.permutation(channel_2.flatten()).reshape(shape)

        # Calculate correlation with randomized channel
        random_corr, _ = pearsonr(channel_1.flatten(), randomized_channel_2.flatten())
        random_correlations.append(random_corr)

    # Calculate p-value: fraction of random correlations >= observed correlation
    p_value = (
        np.sum(np.array(random_correlations) >= observed_correlation) / n_iterations
    )

    return observed_correlation, random_correlations, p_value

We can now run the Costes pixel randomization test and print the pearson's correlation coefficient and the p-value from the test.

**Bonus:** We can also visualize the distribution of the randomized Pearson's correlation coefficients to better understand the significance of our observed correlation.

### Summary

The Python implementation for calculating Pearson's Correlation Coefficient is straightforward and concise, as demonstrated in the code below.

```python
mean_ch1 = np.mean(ch1)
mean_ch2 = np.mean(ch2)
numerator = np.sum((ch1 - mean_ch1) * (ch2 - mean_ch2))
denominator = np.sqrt(np.sum((ch1 - mean_ch1) ** 2) * np.sum((ch2 - mean_ch2) ** 2))
pearson_coefficient = numerator / denominator
```

And it is even easier using already available libraries like `scipy` or `numpy`:

```python
from scipy.stats import pearsonr
pearson_coefficient, p_value = pearsonr(ch1.flatten(), ch2.flatten())
```
```python
import numpy as np
pearson_coefficient = np.corrcoef(ch1.flatten(), ch2.flatten())[0, 1]
```

**Key Considerations for Pearson's Correlation Analysis:**

1. **Region of Interest (ROI) Analysis**: Instead of running Pearson's correlation on the entire image, it is often beneficial to perform **segmentation of the structures you care about** first. Ideally, use a **third independent channel** (such as a nuclear stain or cell membrane marker) to define regions of interest. This approach:
   - Reduces background noise interference
   - Focuses analysis on biologically relevant areas
   - Improves the biological interpretation of results
   - Eliminates correlation artifacts from empty regions

2. **Background Considerations**: Pearson's correlation can be heavily influenced by background pixels. Consider applying background subtraction or flat-field correction before analysis, or use segmentation masks to exclude background regions.

3. **Statistical Validation**: Always validate your results using statistical tests such as the Costes pixel randomization test demonstrated above. This helps assess whether observed correlations are statistically significant or could have occurred by chance.

4. **Comparative Analysis**: Pearson's correlation values should not be interpreted as absolute measures in isolation. Instead, consider them in the context of:
   - Comparisons between different experimental conditions
   - Control vs. treatment groups
   - Different time points or developmental stages
   - Relative changes between conditions are often more meaningful than absolute values

5. **Limitations**: Remember that Pearson's correlation measures **linear relationships** between pixel intensities. It may not capture more complex colocalization patterns and can be sensitive to outliers and intensity variations.

By combining proper image preprocessing, ROI-based analysis, and statistical validation, Pearson's correlation coefficient becomes a powerful tool for quantitative colocalization analysis in fluorescence microscopy.